In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, AutoTokenizer,AutoModelForSeq2SeqLM,BertTokenizerFast
import torch
from transformers import pipeline
import pandas as pd 
import numpy as np
import pyarrow.parquet as pq
from torch.utils.data import Dataset,DataLoader
import regex as re
from googletrans import Translator
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

/home/jaime/Escritorio/AIR/entornos/NLP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-30 09:18:15.453579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730276295.609151   17087 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730276295.652446   17087 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 09:18:16.034945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

# Translate all phrases

## Translate with Google Translate

In [42]:
#tests:
#splits = {'uk': 'data/uk-00000-of-00001-86a756a5ad8b3340.parquet', 'hi': 'data/hi-00000-of-00001-35f39f96bd50e4e4.parquet', 'zh': 'data/zh-00000-of-00001-21350981874ac4f4.parquet', 'ar': 'data/ar-00000-of-00001-2338057ce0eeaf0d.parquet', 'de': 'data/de-00000-of-00001-00ccec8d5e81dbf1.parquet', 'en': 'data/en-00000-of-00001-c54cbb5edf21dd23.parquet', 'ru': 'data/ru-00000-of-00001-6818d57069197d03.parquet', 'am': 'data/am-00000-of-00001-656e19a6f1cc2383.parquet', 'es': 'data/es-00000-of-00001-723005ac428f80c9.parquet'}
#df = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox_test/" + splits["es"])
splits = {'eng_Latn': 'data/en-00000-of-00001.parquet', 
          'rus_Cyrl': 'data/ru-00000-of-00001.parquet', 
          'ukr_Cyrl': 'data/uk-00000-of-00001.parquet', 
          'deu_Latn': 'data/de-00000-of-00001.parquet', 
          'spa_Latn': 'data/es-00000-of-00001.parquet', 
          'amh_Ethi': 'data/am-00000-of-00001.parquet', 
          'zho_Hans': 'data/zh-00000-of-00001.parquet', 
          'arb_Arab': 'data/ar-00000-of-00001.parquet', 
          'hin_Deva': 'data/hi-00000-of-00001.parquet'}
translator = Translator()
all_data=[]
for key, value in splits.items():
    x=0
    final_dataframe = pd.DataFrame(columns=['toxic_sentence_not_translated','neutral_sentence_not_translated','toxic_sentence_translated','neutral_sentence_translated','original_language'])
    df = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + value)
    array = np.array(df)
    for instance in array:
        input=[]
        input.append(instance[0])
        input.append(instance[1])
        toxic_translated = translator.translate(instance[0]).text
        input.append(toxic_translated)
        non_toxic_translated = translator.translate(instance[1]).text
        input.append(non_toxic_translated)
        input.append(key)
        final_dataframe.loc[x]=input
        x+=1
    all_data.append(final_dataframe)
big_dataframe=pd.concat(all_data)
big_dataframe.to_csv('Dataset_tab.csv',sep='\t',index=False)



## Translate with NLLB-200-distilled-600M

In [112]:
#con emojis
class Translator_emojis:
    def __init__(self):
        # Use the original NLLB tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

    def translate(self, src_lan, tgt_lan, text):
        # Split text into words and emojis to handle emojis separately
        words = re.split(r'(\p{So}|\p{Cn})', text, flags=re.UNICODE)
        translated_text = []
        for word in words:
            if word and not re.search(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF]', word):
                # Only translate text segments, skip emojis
                text_with_lang_code = f"{src_lan} {word}"
                inputs = self.tokenizer(text_with_lang_code, return_tensors="pt")
                translated_tokens = self.model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(tgt_lan)
                )
                translated_word = self.tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
            else:
                # If it's an emoji, keep it as is
                print(u'word')
                translated_text += u'\U0001f604'

            # Append translated word or emoji to result
            translated_text.append(translated_word)
        
        return translated_text
    
#Omitiendo emojis
class Translator():
    def __init__(self):
        self.tokenizer=AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        self.model=AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
    def translate(self,src_lan,tgt_lan,text):
        text=f"{src_lan} {text}"
        inputs = self.tokenizer(text, return_tensors="pt")
        translated_tokens = self.model.generate(** inputs, forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(tgt_lan))
        translated=self.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        return translated

In [114]:
translator=Translator()
translator_emojis = Translator_emojis()
print(translator.translate(src_lan='spa_Latn',tgt_lan='eng_Latn',text="Example text with emoji 🤯 and symbol ♡."))
print(translator_emojis.translate(src_lan='spa_Latn',tgt_lan='eng_Latn',text="Example text with emoji 🤯 and symbol ♡."))

Example text with emoji  and symbol ♡.
['Example text with emoji ', '', 'and symbol', '♪ ♪ ♪', "- I 'm not ."]


# Loading Data

In [ ]:
all_languages = pd.read_cs

In [ ]:
class Detox():
    def __init__(self):
        self.base_model_name='facebook/bart-base'
        self.model_name = 's-nlp/bart-base-detox'
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_name)

    def detox(self,input):
        input_ids = self.tokenizer.encode(input, return_tensors='pt')
        output_ids = self.model.generate(input_ids, max_length=50, num_return_sequences=1)
        output_text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return output_text
    
    def train(self, dataloader):